In [12]:
!pip install kgtk==1.0.1

In [1]:
!echo "deb http://downloads.skewed.de/apt bionic main" >> /etc/apt/sources.list
!apt-key adv --keyserver keys.openpgp.org --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-cairo python3-matplotlib
!apt-get install libcairo2-dev 

Executing: /tmp/apt-key-gpghome.xWACPHaJDf/gpg.1.sh --keyserver keys.openpgp.org --recv-key 612DEFB798507F25
gpg: key 612DEFB798507F25: public key "Tiago de Paula Peixoto <tiago@skewed.de>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-u

## Preamble: set up the environment and files used in the assignment (remember to restart runtime)

In [2]:
import io
import os
import subprocess
import sys

import math
import numpy as np
import pandas as pd
from graph_tool.all import *
from IPython.display import display, HTML

from kgtk.configure_kgtk_notebooks import ConfigureKGTK
from kgtk.functions import kgtk, kypher

/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/all.py:39: RuntimeWarning: Error importing draw module, proceeding nevertheless: No module named 'cairo._cairo'
  warnings.warn(msg, RuntimeWarning)


In [3]:
# Parameters

# Folder on local machine where to create the output and temporary folders
input_path = None
output_path = "/tmp/projects"
project_name = "assignment"

The following command will download all the files you  need for the assignment:

In [4]:
files = [
    "all",
    "label",
    "alias",
    "description",
    "external_id",
    "monolingualtext",
    "quantity",
    "string",
    "time",
    "item",
    "wikibase_property",
    "qualifiers",
    "datatypes",
    "p279",
    "p279star",
    "p31",
    "in_degree",
    "out_degree",
    "pagerank_directed",
    "pagerank_undirected"
]
ck = ConfigureKGTK(files)
ck.configure_kgtk(input_graph_path=input_path,
                  output_path=output_path,
                  project_name=project_name)

User home: /root
Current dir: /content
KGTK dir: /
Use-cases dir: //use-cases
--2021-11-21 16:38:57--  https://github.com/usc-isi-i2/kgtk-tutorial-files/raw/main/datasets/arnold/all.tsv.gz
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/usc-isi-i2/kgtk-notebooks/raw/main/datasets/arnold/all.tsv.gz [following]
--2021-11-21 16:38:57--  https://github.com/usc-isi-i2/kgtk-notebooks/raw/main/datasets/arnold/all.tsv.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/datasets/arnold/all.tsv.gz [following]
--2021-11-21 16:38:57--  https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/datasets/arnold/all.tsv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.

The KGTK setup command defines environment variables for all the files so that you can reuse the Jupyter notebook when you install it on your local machine.

In [5]:
ck.print_env_variables()

STORE: /tmp/projects/assignment/temp.assignment/wikidata.sqlite3.db
KGTK_OPTION_DEBUG: false
TEMP: /tmp/projects/assignment/temp.assignment
kypher: kgtk query --graph-cache /tmp/projects/assignment/temp.assignment/wikidata.sqlite3.db
KGTK_GRAPH_CACHE: /tmp/projects/assignment/temp.assignment/wikidata.sqlite3.db
kgtk: kgtk
OUT: /tmp/projects/assignment
KGTK_LABEL_FILE: /root/isi-kgtk-tutorial/input/labels.en.tsv.gz
USE_CASES_DIR: //use-cases
GRAPH: /root/isi-kgtk-tutorial/input
EXAMPLES_DIR: //examples
all: /root/isi-kgtk-tutorial/input/all.tsv.gz
label: /root/isi-kgtk-tutorial/input/labels.en.tsv.gz
alias: /root/isi-kgtk-tutorial/input/aliases.en.tsv.gz
description: /root/isi-kgtk-tutorial/input/descriptions.en.tsv.gz
external_id: /root/isi-kgtk-tutorial/input/claims.external-id.tsv.gz
monolingualtext: /root/isi-kgtk-tutorial/input/claims.monolingualtext.tsv.gz
quantity: /root/isi-kgtk-tutorial/input/claims.quantity.tsv.gz
string: /root/isi-kgtk-tutorial/input/claims.string.tsv.gz
time

In [6]:
%%time
ck.load_files_into_cache()

kgtk query --graph-cache /tmp/projects/assignment/temp.assignment/wikidata.sqlite3.db -i "/root/isi-kgtk-tutorial/input/all.tsv.gz" --as all  -i "/root/isi-kgtk-tutorial/input/labels.en.tsv.gz" --as label  -i "/root/isi-kgtk-tutorial/input/aliases.en.tsv.gz" --as alias  -i "/root/isi-kgtk-tutorial/input/descriptions.en.tsv.gz" --as description  -i "/root/isi-kgtk-tutorial/input/claims.external-id.tsv.gz" --as external_id  -i "/root/isi-kgtk-tutorial/input/claims.monolingualtext.tsv.gz" --as monolingualtext  -i "/root/isi-kgtk-tutorial/input/claims.quantity.tsv.gz" --as quantity  -i "/root/isi-kgtk-tutorial/input/claims.string.tsv.gz" --as string  -i "/root/isi-kgtk-tutorial/input/claims.time.tsv.gz" --as time  -i "/root/isi-kgtk-tutorial/input/claims.wikibase-item.tsv.gz" --as item  -i "/root/isi-kgtk-tutorial/input/claims.wikibase-property.tsv.gz" --as wikibase_property  -i "/root/isi-kgtk-tutorial/input/qualifiers.tsv.gz" --as qualifiers  -i "/root/isi-kgtk-tutorial/input/metadata.pr

# About this assignment.
This assignment is based on https://github.com/usc-isi-i2/kgtk-notebooks/tree/main/tutorial. If you have any questions or doubts, it is encouraged to look how the tutorial performs the different operations.

Additional information can be found in https://kgtk.readthedocs.io/

## Simple graph statistics

Let's calculate first some statistics about the KG. Count the number of instances:


In [7]:
%%time
kgtk("""
    query -i all
        # TO DO
        --match '(instance)-[:P31]->(class)'
        --return 'count(distinct instance) as n_instances'
""")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DtypeWarning: Columns (2,4) have mixed types.Specify dtype option on import or set low_memory=False.
  


CPU times: user 22.4 s, sys: 8.86 s, total: 31.2 s
Wall time: 44 s


,node1,label,node2,id,node2;wikidatatype
0,P10,alias,'gif'@en,P10-alias-en-282226-0,NaN
1,P10,alias,'animation'@en,P10-alias-en-2f86d8-0,NaN
2,P10,alias,'media'@en,P10-alias-en-c1427e-0,NaN
3,P10,alias,'trailer (Commons)'@en,P10-alias-en-c61ab1-0,NaN
4,P1001,alias,'belongs to jurisdiction'@en,P1001-alias-en-0dd7ce-0,NaN
...,...,...,...,...,...
2654665,Q999763-P856-e3d19b-5d94f88b-0,P407,Q150,Q999763-P856-e3d19b-5d94f88b-0-P407-Q150-0,NaN
2654666,Q999763-P8687-97dcb7-868501bd-0,P585,^2018-05-10T00:00:00Z/11,Q999763-P8687-97dcb7-868501bd-0-P585-c40c2a-0,NaN
2654667,Q999763-P8687-97dcb7-868501bd-0,P6552,115711568,Q999763-P8687-97dcb7-868501bd-0-P6552-c54a18-0,NaN
2654668,Q999763-P8687-d9bd38-376d13b0-0,P585,^2021-01-05T00:00:00Z/11,Q999763-P8687-d9bd38-376d13b0-0-P585-db4da1-0,NaN


Now, count the number of distinct properties: 


In [20]:
%%time
kgtk("""
    query -i all
    --match '(instance)-[prop {label: property}]->(class)'
    --return 'count(distinct property) as n_prop'
""")

CPU times: user 16 ms, sys: 27.4 ms, total: 43.4 ms
Wall time: 1.92 s


,n_prop
0,3874


Now, let's count the frequency of those properties. That is, how many instances we can find with each property

In [24]:
%%time
kgtk("""

   query -i all
         --match '(instance)-[prop {label: property}]->(class)'
         --return 'prop.label, count(distinct instance) as prop_frec'
""")

CPU times: user 160 ms, sys: 48.2 ms, total: 208 ms
Wall time: 4.39 s


,label,prop_frec
0,P1001,2561
1,P1003,20
2,P1004,327
3,P1005,767
4,P1006,3538
...,...,...
3869,alias,34151
3870,data_type,15
3871,datatype,3864
3872,description,59862


## Simple queries
Some of these queries are simple and will run in the Wikidata endpoint. 
Try both of them using SPARQL and Kypher

In [32]:
# Which actors has Schwarzenegger worked with throughout his career? (Print also the movie)

# (in SPARQL)
# TO DO 
#SELECT DISTINCT ?actor ?movie ?actorLabel ?movieLabel WHERE {
#   ?movie wdt:P161 wd:Q2685.
#   ?movie wdt:P161 ?actor.
#   FILTER (?actor != wd:Q2685)
#   SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
# In Kypher:
kgtk("""
    query -i all
        --match '(:Q2685)<-[:P161]-(movie)-[:P161]->(actor)'
        --where 'actor != "Q2685"'
        --return 'actor, movie'
    / add-labels
""")

,node2,node1,node2;label,node1;label
0,Q106997,Q110397,'Jamie Lee Curtis'@en,'True Lies'@en
1,Q110374,Q110397,'Bill Paxton'@en,'True Lies'@en
2,Q3181135,Q110397,'John Bruno'@en,'True Lies'@en
3,Q3436301,Q110397,'Bobby Shriver'@en,'True Lies'@en
4,Q1252523,Q15140437,'Douglas Smith'@en,'Terminator Genisys'@en
...,...,...,...,...
204,Q720530,Q858840,'Noah Emmerich'@en,'Last Action Hero'@en
205,Q781151,Q858840,'Austin O'Brien'@en,'Last Action Hero'@en
206,Q82222,Q858840,'Little Richard'@en,'Last Action Hero'@en
207,Q83484,Q858840,'Anthony Quinn'@en,'Last Action Hero'@en


In [38]:
# How many awards does Schwarzenegger have?

# SPARQL:
# TO DO
#SELECT (COUNT(?award) AS ?count_awards) WHERE {
#   wd:Q2685 wdt:P166 ?award
#}
# Kypher:
kgtk("""
    query -i all
        --match '(:Q2685)-[:P166]->(award)'
        
        --return 'count(award) as award'
   
""")


,award
0,16


In [40]:
# Retrieve at least two members of Schwarzenegger's political party. Make sure only persons are returned
# SPARQL:
# TO DO
#SELECT ?name ?nameLabel ?polit_party ?polit_partyLabel WHERE {
#  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
#  wd:Q2685 wdt:P102 ?polit_party.
#  OPTIONAL { ?name wdt:P102 ?polit_party. }
#}
# Kypher:
kgtk("""
    query -i all 
        --match '(:Q2685)-[:P102]->(polit_party)<-[:P102]-(person)-[:P31]->(:Q5)'
        --return 'person, polit_party'
    / add-labels
""")

,node1,node2,node1;label,node2;label
0,Q102711,Q29468,'Dennis Hopper'@en,'Republican Party'@en
1,Q103939,Q29468,'Charlie Sheen'@en,'Republican Party'@en
2,Q104161,Q29468,'Harry Warner'@en,'Republican Party'@en
3,Q1065770,Q29468,'Charles Phelps Taft'@en,'Republican Party'@en
4,Q1072969,Q29468,'Chill Wills'@en,'Republican Party'@en
...,...,...,...,...
663,Q952884,Q29468,'Richard A. Ballinger'@en,'Republican Party'@en
664,Q9582,Q29468,'Gerald Ford'@en,'Republican Party'@en
665,Q9588,Q29468,'Richard Nixon'@en,'Republican Party'@en
666,Q9916,Q29468,'Dwight D. Eisenhower'@en,'Republican Party'@en


In [50]:
# What are the properties that describe an artist?

# In theory this one is heavy on Wikidata

# SPARQL: 
# TO DO
#SELECT DISTINCT ?prop WHERE{
#  ?x rdfs:subClassOf* wd:Q483501.
#  ?a wdt:P106 ?x.
#  ?a ?prop ?o
#}

# In Kypher:
kgtk("""
query -i all
    --match '(instance)<-[prop {label: property}]-(person)-[:P106]->(artist)-[:P279]->(:Q483501)'
    
    --return 'property'
    
""")


,label
0,alias
1,P1006
2,P1207
3,P1220
4,P1258
...,...
40355,label
40356,Pin_degree
40357,Pout_degree
40358,Pdirected_pagerank


In [51]:
# And a film director?
# SPARQL: 
# TO DO
#SELECT DISTINCT ?prop WHERE {
#   ?x rdfs:subClassOf* wd:Q2526255.
#   ?a wdt:P106 ?x.
#   ?a ?property ?o.
#}
# In Kypher:
kgtk("""
 query -i all
        --match '(instance)<-[l {label: property}]-(element)-[:P106]->(class)-[:P279]->(:Q2526255)'
        --return 'distinct property'
""")

,label
0,alias
1,P1006
2,P1207
3,P1220
4,P1258
...,...
315,P4114
316,P7704
317,P8511
318,P3495


In [ ]:
# Embeddings. Run the noebook https://colab.research.google.com/drive/1A55l10voA4jnjoju3fojJWY3buLfaR4i?usp=sharing. 
# Which are the top 10 similar entities to Schwarzenegger? (list below) 
# TO DO

## Network analysis
Print all the paths between Schwarzenegger and Trump

## Note that **you have to create a file `paths.tsv` with the node pairs you want to find the paths for. Upload it in the "content" folder**


In [52]:
%%bash
cat <<EOF >$TEMP/path-query.tsv
node1	node2	label
Q2685	Q22686	path

bash: line 3: warning: here-document at line 1 delimited by end-of-file (wanted `EOF')


In [53]:
kgtk("""
    add-labels -i $TEMP/path-query.tsv
""")

,node1,node2,label,node1;label,node2;label
0,Q2685,Q22686,path,'Arnold Schwarzenegger'@en,'Donald Trump'@en


In [54]:
# Calculate all the paths between Trump and Schwarzenegger (max hops: 3)
# TO DO    
kgtk("""
paths -i all 
        --path-file $TEMP/path-query.tsv 
        --statistics-only True 
        --max_hops 3
""")

/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/all.py:39: RuntimeWarning: Error importing draw module, proceeding nevertheless: No module named 'cairo._cairo'
  warnings.warn(msg, RuntimeWarning)



,node1,label,node2,id
0,p0,0,Q2685-P102-Q29468-570aa385-0,p0-0-0
1,p0,1,Q29468-P17-Q30-d4884fd2-0,p0-1-1
2,p0,2,Q30-P6-Q22686-a7f750ba-0,p0-2-2
3,p1,0,Q2685-P1038-Q24004771-86e520f2-0,p1-0-3
4,p1,1,Q24004771-P27-Q30-23d231a3-0,p1-1-4
...,...,...,...,...
90,p30,1,Q39072454-P495-Q30-8a090b82-0,p30-1-90
91,p30,2,Q30-P6-Q22686-a7f750ba-0,p30-2-91
92,p31,0,Q2685-P800-Q858840-e01abfe3-0,p31-0-92
93,p31,1,Q858840-P495-Q30-7b4eadcb-0,p31-1-93


In [55]:
# Retrieve all the family of Schwarzenegger (child/father/mother/sibling/spouse relationships)
# TO DO  
kgtk("""
 reachable-nodes -i all
        --root Q2685
        --props P40 P3373 P26 P451 P22 P25
    
""")

,node1,label,node2
0,Q2685,reachable,Q3288486
1,Q2685,reachable,Q23800370
2,Q2685,reachable,Q96076900
3,Q2685,reachable,Q38196234
4,Q2685,reachable,Q24004771
...,...,...,...
67,Q2685,reachable,Q43100988
68,Q2685,reachable,Q901541
69,Q2685,reachable,Q23800185
70,Q2685,reachable,Q75494768


In [56]:
#Corregir esto que parece fallar
# What are the 10 most relevant actors (pagerank) in the graph? (Use graph-statistics command to calculate page rank, and then filter only actors)
# TO DO  
kgtk("""
graph-statistics -i "$item" -o /tmp/projects/assignment/metadata.pagerank.undirected.tsv.gz 
    --compute-pagerank True 
    --compute-hits False 
    --page-rank-property Pdirected_pagerank 
    --vertex-in-degree-property Pindegree
    --vertex-out-degree-property Poutdegree
    --output-degrees True 
    --output-pagerank True 
    --output-hits False \
    --output-statistics-only 
    --undirected True 
    --log-file /tmp/projects/assignment/metadata.pagerank.undirected.summary.txt
""")

/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/all.py:39: RuntimeWarning: Error importing draw module, proceeding nevertheless: No module named 'cairo._cairo'
  warnings.warn(msg, RuntimeWarning)



In [57]:
# TO DO: Hint: do the query after calculating the pagerank. See https://github.com/usc-isi-i2/kgtk-notebooks/blob/main/tutorial/06-kg-network-analysis.ipynb for inspiration
kgtk("""
query  -i all -i /tmp/projects/assignment/metadata.pagerank.undirected.tsv.gz
        --match '
            all: (person)-[:P106]->(:Q33999),
            pagerank: (person)-[:P_pagerank]->(pagerank)'
        --return 'person as node1, pagerank as node2'
        --order-by 'cast(pagerank, float) desc'
        
""")

,node1,node2
